In [1]:
import re
import pandas as pd

In [35]:
###
# Get the courses from a file, into a dict.
###

courses = {}

with open('./pages/HIS-2019-fall.txt', 'r') as course_file:
    # Read course file into one big string
    data = course_file.read()
    
    # Break down to courses
    data = data.split('Collapse section ')
    
    # Remove erroneous space items
    del data[0]
    
    # Key the courses into the `courses` object
    for course in data:
        courses[course[0:8]] = course

In [36]:
def handle_section(section):
    sectionCode = section[0].split('-')[0]
    
    sectionData = {
        "admin": {},
        "days": []
    }
    
    ## Handling admin stuff
    sectionData["admin"]["code"] = sectionCode
    sectionData["admin"]["type"] = section[0].split('-')[1].split('\n')[0]
    sectionData["admin"]["duration"] = section[0].split('-')[1].split('\n')[1]
    sectionData["admin"]["isOpen"] = 'Open' in section[5]
    
    ## Handling days
    
    ### Helper function to extract datetime details
    def handle_section_datetime(datetime):
        return {
            "weekday": datetime[:2],
            "time.start": datetime[3:].split(' - ')[0],
            "time.end": datetime[3:].split(' - ')[1]
        }

    ### Extract the relevant details in a very hacky way :)
    def mini_day_extractor(sectionToHandle, index):
        return {
            **handle_section_datetime(sectionToHandle[1].split('\n')[index]),
            "location": sectionToHandle[2].split('\n')[index],
            "prof": sectionToHandle[3].split('\n')[index],
            "date.start": sectionToHandle[4].split('\n')[index].split(' - ')[0],
            "date.end": sectionToHandle[4].split('\n')[index].split(' - ')[1]
        }
    
    ### Given the number of days (counted by the number of `\n` in the datetime string...), extract them
    for index in range(0, section[1].count('\n') + 1):
        sectionData["days"].append(mini_day_extractor(section, index))
            
    return {sectionCode: sectionData}

In [37]:
###
# Get the sections from each course, adding as a list to the dict item.
###

for courseCode, courseStr in courses.items():
    sections = {}
    
    if ' \n\t\t\n\t\n\t\t\n\t\n\t\t\t\n\t\n  \tSection \tDays & Times \tRoom \tInstructor \tMeeting Dates \tStatus\nDetails\n\t\n' in courseStr:
        sectionContainer = courseStr.split(' \n\t\t\n\t\n\t\t\n\t\n\t\t\t\n\t\n  \tSection \tDays & Times \tRoom \tInstructor \tMeeting Dates \tStatus\nDetails\n\t\n')[1]
    else:
        sectionContainer = courseStr.split(' \n\t\t\n\t\n\t\t\n\t\n\t\t\n\t\n  \tSection \tDays & Times \tRoom \tInstructor \tMeeting Dates \tStatus\nDetails\n\t\n')[1]
    
    sectionData = sectionContainer.split('\n\t\n\t\t\n\t\n  \tSection \tDays & Times \tRoom \tInstructor \tMeeting Dates \tStatus\nDetails\n\t\n')
    
    for section in sectionData:
        sectionInfo = handle_section(section.split('\n\t\n'))
        
        sections.update(sectionInfo)
        
    courses[courseCode] = sections
    
## (sometimes this errors out with "list index out of range". it's all hacky anyway. power through!)

In [38]:
## Clean badly processed courses :)
coursesToDelete = {}

for courseCode, courseObj in courses.items():
    if isinstance(courseObj, str):
        coursesToDelete[courseCode] = courseObj
        
for course in coursesToDelete:
    courses.pop(course, None)
        
print(str(len(coursesToDelete)) + " courses removed due to badly formed data.")

0 courses removed due to badly formed data.


In [39]:
courses

{'HIS 1101': {'A00': {'admin': {'code': 'A00',
    'duration': 'FullSess.',
    'isOpen': True,
    'type': 'LEC'},
   'days': [{'date.end': '2019-12-03',
     'date.start': '2019-09-04',
     'location': '161 Louis Pasteur (CBY) C03',
     'prof': 'Damien-Claude Bélanger',
     'time.end': '14:20',
     'time.start': '13:00',
     'weekday': 'We'},
    {'date.end': '2019-12-03',
     'date.start': '2019-09-04',
     'location': '550 Cumberland (TBT) 333',
     'prof': 'Damien-Claude Bélanger',
     'time.end': '12:50',
     'time.start': '11:30',
     'weekday': 'Fr'}]}},
 'HIS 1110': {'A00': {'admin': {'code': 'A00',
    'duration': 'FullSess.',
    'isOpen': True,
    'type': 'LEC'},
   'days': [{'date.end': '2019-12-03',
     'date.start': '2019-09-04',
     'location': '125 University (MNT) 202',
     'prof': 'Eric Allina',
     'time.end': '12:50',
     'time.start': '11:30',
     'weekday': 'Mo'},
    {'date.end': '2019-12-03',
     'date.start': '2019-09-04',
     'location': '

In [40]:
## Convert the courses object to a dataframe
df = pd.DataFrame.from_dict(courses, orient='index')
df = pd.DataFrame(df.stack())

df

0
HIS 1101 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 1110 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 1111 B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...
         WB00  {'admin': {'code': 'WB00', 'type': 'LEC', 'dur...
HIS 1500 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 1501 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 1520 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2100 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
         B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...
HIS 2101 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2103 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2105 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2129 B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...
HIS 2152 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2160 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2171 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2336 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2341 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2362 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2500 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2501 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2551 B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...
HIS 2552 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2576 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2597 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2707 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2735 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2764 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 3105 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 3110 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
...                                                          ...
HIS 3154 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 3301 B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...
         C00   {'admin': {'code': 'C00', 'type': 'LEC', 'dura...
HIS 3315 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 3355 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 3397 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 3550 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 3728 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 3765 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 4100 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
         B00   {'admin': {'code': 'B00', 'type': 'SEM', 'dura...
HIS 4135 B00   {'admin': {'code': 'B00', 'type': 'SEM', 'dura...
         C00   {'admin': {'code': 'C00', 'type': 'SEM', 'dura...
HIS 4150 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 4192 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 4301 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 4361 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 4362 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 4380 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 4397 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 4535 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 4551 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 4762 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 4770 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 5103 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 5503 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 6332 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 7338 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 7738 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 8900 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...

[63 rows x 1 columns]

In [41]:
## Unpack the `data` column
df = df.reset_index()
df.columns = ['course', 'code', 'data']
df

df2 = df.join(pd.io.json.json_normalize(df['data']))
df2 = df2.set_index(['course', 'code'])

df2

data admin.code  \
course   code                                                                 
HIS 1101 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 1110 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 1111 B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...        B00   
         WB00  {'admin': {'code': 'WB00', 'type': 'LEC', 'dur...       WB00   
HIS 1500 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 1501 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 1520 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 2100 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
         B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...        B00   
HIS 2101 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 2103 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 2105 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 2129 B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...        B00   
HIS 2152 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 2160 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 2171 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 2336 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 2341 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 2362 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 2500 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 2501 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 2551 B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...        B00   
HIS 2552 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 2576 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 2597 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 2707 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 2735 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 2764 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 3105 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 3110 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
...                                                          ...        ...   
HIS 3154 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 3301 B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...        B00   
         C00   {'admin': {'code': 'C00', 'type': 'LEC', 'dura...        C00   
HIS 3315 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 3355 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 3397 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 3550 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 3728 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 3765 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 4100 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...        A00   
         B00   {'admin': {'code': 'B00', 'type': 'SEM', 'dura...        B00   
HIS 4135 B00   {'admin': {'code': 'B00', 'type': 'SEM', 'dura...        B00   
         C00   {'admin': {'code': 'C00', 'type': 'SEM', 'dura...        C00   
HIS 4150 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...        A00   
HIS 4192 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...        A00   
HIS 4301 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 4361 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 4362 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...        A00   
HIS 4380 A00   {'admin': {'code

In [42]:
## Unpack the `days` column

### Convert the days column from an object to a string
import ast

days_as_string = df2.astype({'days': str}).reset_index()['days'].apply(ast.literal_eval)
days_as_string

### Unpack the column. The column contains lists of objects with consistent keys, so each object becomes its own row
days_by_section = pd.concat([pd.DataFrame(x) for x in days_as_string], keys=days_as_string.index)
days_by_section

### Join the expanded rows with their original courses, dropping the now-unused columns
df3 = df2.reset_index().join(days_by_section.reset_index(1, drop=True))
df3 = df3.set_index(['course', 'code']).drop('data', 1).drop('days', 1)

df3

admin.code admin.duration  admin.isOpen admin.type    date.end  \
course   code                                                                  
HIS 1101 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
HIS 1110 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
HIS 1111 B00         B00      FullSess.          True        LEC  2019-12-03   
         B00         B00      FullSess.          True        LEC  2019-12-03   
         WB00       WB00      FullSess.          True        LEC  2019-12-03   
HIS 1500 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
HIS 1501 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
HIS 1520 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
HIS 2100 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
         B00         B00      FullSess.          True        LEC  2019-12-03   
         B00         B00      FullSess.          True        LEC  2019-12-03   
HIS 2101 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
HIS 2103 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
HIS 2105 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
HIS 2129 B00         B00      FullSess.          True        LEC  2019-12-03   
HIS 2152 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
HIS 2160 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
HIS 2171 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
...                  ...            ...           ...        ...         ...   
HIS 3397 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
HIS 3550 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
HIS 3728 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
HIS 3765 A00         A00      FullSess.          True        LEC  2019-12-03   
HIS 4100 A00         A00      FullSess.          True        SEM  2019-12-03   
         B00         B00      FullSess.          True        SEM  2019-12-03   
HIS 4135 B00         B00      FullSess.          True        SEM  2019-12-03   
         C00         C00      FullSess.          True        SEM  2019-12-03   
HIS 4150 A00         A00      FullSess.          True        SEM  2019-12-03   
         A00         A00      FullSess.          True        SEM  2019-12-03   
HIS 4192 A00         A00      FullSess.          True        SEM  2019-12-03   
HIS 4301 A00         A00      FullSess.          True        LEC  2019-12-03   
HIS 4361 A00         A00      FullSess.          True        LEC  2019-12-03   
         A00         A00      FullSess.          True        LEC  2019-12-03   
HIS 4362 A00  

In [43]:
## Reorganize columns
courses_by_section_by_day = df3[[
    'admin.duration',
    'admin.isOpen',
    'admin.type',
    'prof',
    'weekday',
    'time.start',
    'time.end',
    'location',
    'date.start',
    'date.end'
]]
courses_by_section_by_day

admin.duration  admin.isOpen admin.type                    prof  \
course   code                                                                   
HIS 1101 A00       FullSess.          True        LEC  Damien-Claude Bélanger   
         A00       FullSess.          True        LEC  Damien-Claude Bélanger   
HIS 1110 A00       FullSess.          True        LEC             Eric Allina   
         A00       FullSess.          True        LEC       Meredith Terretta   
HIS 1111 B00       FullSess.          True        LEC                   Staff   
         B00       FullSess.          True        LEC                   Staff   
         WB00      FullSess.          True        LEC                   Staff   
HIS 1500 A00       FullSess.          True        LEC             Kouky Fianu   
         A00       FullSess.          True        LEC             Kouky Fianu   
HIS 1501 A00       FullSess.          True        LEC    Jean-François Lozier   
         A00       FullSess.          True        LEC    Jean-François Lozier   
HIS 1520 A00       FullSess.          True        LEC          Corinne Gaudin   
         A00       FullSess.          True        LEC          Corinne Gaudin   
HIS 2100 A00       FullSess.          True        LEC                   Staff   
         A00       FullSess.          True        LEC                   Staff   
         B00       FullSess.          True        LEC                   Staff   
         B00       FullSess.          True        LEC                   Staff   
HIS 2101 A00       FullSess.          True        LEC                   Staff   
         A00       FullSess.          True        LEC                   Staff   
HIS 2103 A00       FullSess.          True        LEC                   Staff   
         A00       FullSess.          True        LEC                   Staff   
HIS 2105 A00       FullSess.          True        LEC           Pierre Anctil   
         A00       FullSess.          True        LEC           Pierre Anctil   
HIS 2129 B00       FullSess.          True        LEC                   Staff   
HIS 2152 A00       FullSess.          True        LEC            Galen Perras   
         A00       FullSess.          True        LEC            Galen Perras   
HIS 2160 A00       FullSess.          True        LEC         Ryme Seferdjeli   
         A00       FullSess.          True        LEC         Ryme Seferdjeli   
HIS 2171 A00       FullSess.          True        LEC  Alberto Florez-Malagon   
         A00       FullSess.          True        LEC  Alberto Florez-Malagon   
...                      ...           ...        ...                     ...   
HIS 3397 A00       FullSess.          True        LEC                   Staff   
         A00       FullSess.          True        LEC                   Staff   
HIS 3550 A00       FullSess.          True        LEC                   Staff   
         A00       FullSess.          True        LEC                   Staff   
HIS 3728 A00       FullSess.          True        LEC                   Staff   
         A00       FullSess.          True        LEC                   Staff   
HIS 3765 A00       FullSess.          True        LEC                   Staff   
HIS 4100 A00       FullSess.          True        SEM                   Staff   
         B00       FullSess.          True        SEM                   Staff   
HIS 4135 B00       FullSess.          True        SEM                   Staff   
         C00       FullSess.          True        SEM                   Staff   
HIS 4150 A00       FullSess.          True        SEM                   Staff   
         A00       FullSess.          True        SEM                   Staff   
HIS 4192 A00       FullSess.          True        SEM                   Staff   
HIS 4301 A00       FullSess.          True        LEC                   Staff   
HIS 4361 A00       FullSess.          True        LEC                   Staff   
         A00       FullSess.          True   

In [44]:
## Find all courses taught by “Staff”
courses_by_section_by_day[courses_by_section_by_day['prof'].str.contains('Staff')]

admin.duration  admin.isOpen admin.type   prof weekday  \
course   code                                                          
HIS 1111 B00       FullSess.          True        LEC  Staff      Tu   
         B00       FullSess.          True        LEC  Staff      Fr   
         WB00      FullSess.          True        LEC  Staff      TB   
HIS 2100 A00       FullSess.          True        LEC  Staff      Tu   
         A00       FullSess.          True        LEC  Staff      Fr   
         B00       FullSess.          True        LEC  Staff      Mo   
         B00       FullSess.          True        LEC  Staff      Th   
HIS 2101 A00       FullSess.          True        LEC  Staff      Tu   
         A00       FullSess.          True        LEC  Staff      Fr   
HIS 2103 A00       FullSess.          True        LEC  Staff      We   
         A00       FullSess.          True        LEC  Staff      Fr   
HIS 2129 B00       FullSess.          True        LEC  Staff      Mo   
HIS 2336 A00       FullSess.          True        LEC  Staff      We   
         A00       FullSess.          True        LEC  Staff      Fr   
HIS 2341 A00       FullSess.          True        LEC  Staff      Mo   
         A00       FullSess.          True        LEC  Staff      We   
HIS 2500 A00       FullSess.          True        LEC  Staff      Tu   
         A00       FullSess.          True        LEC  Staff      Fr   
HIS 2552 A00       FullSess.          True        LEC  Staff      Tu   
         A00       FullSess.          True        LEC  Staff      Th   
HIS 2576 A00       FullSess.          True        LEC  Staff      We   
HIS 2597 A00       FullSess.          True        LEC  Staff      Th   
HIS 2707 A00       FullSess.          True        LEC  Staff      Mo   
         A00       FullSess.          True        LEC  Staff      We   
HIS 2735 A00       FullSess.          True        LEC  Staff      Tu   
         A00       FullSess.          True        LEC  Staff      Th   
HIS 2764 A00       FullSess.          True        LEC  Staff      Mo   
         A00       FullSess.          True        LEC  Staff      We   
HIS 3105 A00       FullSess.          True        LEC  Staff      Tu   
         A00       FullSess.          True        LEC  Staff      Th   
...                      ...           ...        ...    ...     ...   
HIS 3397 A00       FullSess.          True        LEC  Staff      Mo   
         A00       FullSess.          True        LEC  Staff      Th   
HIS 3550 A00       FullSess.          True        LEC  Staff      Mo   
         A00       FullSess.          True        LEC  Staff      We   
HIS 3728 A00       FullSess.          True        LEC  Staff      We   
         A00       FullSess.          True        LEC  Staff      Fr   
HIS 3765 A00       FullSess.          True        LEC  Staff      Tu   
HIS 4100 A00       FullSess.          True        SEM  Staff      Tu   
         B00       FullSess.          True        SEM  Staff      Th   
HIS 4135 B00       FullSess.          True        SEM  Staff      Mo   
         C00       FullSess.          True        SEM  Staff      Tu   
HIS 4150 A00       FullSess.          True        SEM  Staff      Mo   
         A00       FullSess.          True        SEM  Staff      We   
HIS 4192 A00       FullSess.          True        SEM  Staff      We   
HIS 4301 A00       FullSess.          True        LEC  Staff      Mo   
HIS 4361 A00       FullSess.          True        LEC  Staff      Tu   
         A00       FullSess.          True        LEC  Staff      Th   
HIS 4362 A00       FullSess.          True        LEC  Staff      Tu   
HIS 4380 A00       FullSess.          True        SEM  Staff      Th   
HIS 4397 A00       FullSess.          True        SEM  Staff      We   
HIS 4535 A00       FullSess.          True        SEM  Staff      Fr   
HIS 4551 A00       FullSess.          True        SEM  Staff      Tu   
HIS 4762 A00       FullSess.          True        LEC 

In [46]:
course_descriptions = pd.read_csv('data/courses/HIS.csv').set_index('code')

course_descriptions

,credits,year,language,title,description,extraDetails
code,,,,,,
HIS 1100,3,1,English,History Labs,"Using a hand-on approach, based on primary sou...","['Course Component: Lecture, Lecture']"
HIS 1101,3,1,English,The Making of Canada,"Survey of the political, social and cultural e...",['Course Component: Lecture']
HIS 1110,3,1,English,Introduction to Global History,A course of general interest focused on human ...,['Course Component: Lecture']
HIS 1111,3,1,English,The Twentieth-Century World from 1945,A course of general interest which focuses on ...,['Course Component: Lecture']
HIS 1120,3,1,English,What Is Europe? (16th-21th Century),Long term study of the changing nature of Euro...,['Course Component: Lecture']
HIS 1500,3,1,French,Laboratoires d'histoire,"Par son orientation pratique, fondée sur l'uti...","['Volet : Cours magistral, Cours magistral']"
HIS 1501,3,1,French,La formation du Canada,"Survol de l'évolution politique, sociale et cu...",['Volet : Cours magistral']
HIS 1510,3,1,French,Initiation à l'histoire globale,Cours d'intérêt général axé sur les migrations...,['Volet : Cours magistral']
HIS 1511,3,1,French,Le monde au XXe siècle depuis 1945,Cours d'intérêt général portant sur les questi...,['Volet : Cours magistral']


In [47]:
described_courses_by_section_by_day = pd.merge(courses_by_section_by_day.reset_index(1), course_descriptions, left_index=True, right_index=True)

described_courses_by_section_by_day = described_courses_by_section_by_day.reset_index().set_index(['index', 'code'])
described_courses_by_section_by_day.index.names = ['course', 'section']

described_courses_by_section_by_day

admin.duration  admin.isOpen admin.type  \
course   section                                           
HIS 1101 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 1110 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 1111 B00          FullSess.          True        LEC   
         B00          FullSess.          True        LEC   
         WB00         FullSess.          True        LEC   
HIS 1500 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 1501 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 1520 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 2100 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
         B00          FullSess.          True        LEC   
         B00          FullSess.          True        LEC   
HIS 2101 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 2103 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 2105 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 2129 B00          FullSess.          True        LEC   
HIS 2152 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 2160 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 2171 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
...                         ...           ...        ...   
HIS 3397 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 3550 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 3728 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 3765 A00          FullSess.          True        LEC   
HIS 4100 A00          FullSess.          True        SEM   
         B00          FullSess.          True        SEM   
HIS 4135 B00          FullSess.          True        SEM   
         C00          FullSess.          True        SEM   
HIS 4150 A00          FullSess.          True        SEM   
         A00          FullSess.          True        SEM   
HIS 4192 A00          FullSess.          True        SEM   
HIS 4301 A00          FullSess.          True        LEC   
HIS 4361 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 4362 A00          FullSess.          True        LEC   
HIS 4380 A00          FullSess.          True        SEM   
HIS 4397 A00          FullSess.          True        SEM   
HIS 4535 A00          FullSess.          True        SEM   
HIS 4551 A00          FullSess.          True        SEM   
HIS 4762 A00          FullSess.          True        LEC   
HIS 4770 A00          FullSess.          True        SEM   
HIS 5103 A00          FullSess.          True        SEM   
HIS 5503 A00          FullSess.          True        SEM   
HIS 6332 A00          FullSess.          True        SEM   
HIS 7338 A00          FullSess.          True        SEM   
HIS 7738 A00          FullSess.          True        SEM   
HIS 8900 A00          FullSess.          True        SEM   

                                    prof weekday time.start time.end  \
course   section                                                       
HIS 1101 A00      Damien-Claude Bélanger      We      13:00    14:20   
         A00      Da

In [54]:
described_courses_by_section_by_day.query('course == "HIS 1110"')

admin.duration  admin.isOpen admin.type               prof  \
course   section                                                              
HIS 1110 A00          FullSess.          True        LEC        Eric Allina   
         A00          FullSess.          True        LEC  Meredith Terretta   

                 weekday time.start time.end                  location  \
course   section                                                         
HIS 1110 A00          Mo      11:30    12:50  125 University (MNT) 202   
         A00          Th      13:00    14:20  125 University (MNT) 202   

                  date.start    date.end  credits  year language  \
course   section                                                   
HIS 1110 A00      2019-09-04  2019-12-03        3     1  English   
         A00      2019-09-04  2019-12-03        3     1  English   

                                           title  \
course   section                                   
HIS 1110 A00      Introduction to Global History   
         A00      Introduction to Global History   

                                                        description  \
course   section                                                      
HIS 1110 A00      A course of general interest focused on human ...   
         A00      A course of general interest focused on human ...   

                                   extraDetails  
course   section                                 
HIS 1110 A00      ['Course Component: Lecture']  
         A00      ['Course Component: Lecture']